In [ ]:
#writefile Error_Analisys_Lib.py 

import bempp.api, numpy as np
from math import pi
import os

from bempp.api.operators.potential import laplace as lp
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

from Grid_Maker_R2 import *
from Mesh_Ref      import *
from quadrature    import *





def charges_loader( path , mol_name ):
    '''
    path    : Molecule/{mol_name} --- Directory where the files are saved
    mol_name: Abreviated name of the molecule
    '''
    q, x_q = np.empty(0), np.empty((0,3))
    pqr_file = os.path.join(path,mol_name+'.pqr')
    charges_file = open( pqr_file , 'r').read().split('\n')

    for line in charges_file:
        line = line.split()
        if len(line)==0: continue
        if line[0]!='ATOM': continue
        q = np.append( q, float(line[8]))
        x_q = np.vstack( ( x_q, np.array(line[5:8]).astype(float) ) )   
    
    return x_q , q

def zero_i(x, n, domain_index, result):
    '''
    Builds 0 vector.
    '''
    global q,x_q,ep_m,C
    
    result[:] = 0

def u_s_G(x,n,domain_index,result):
    '''
    The singular component (Coulomb potential) 
    '''
    global q,x_q,ep_m,C
    result[:] = C / (4.*np.pi*ep_m)  * np.sum( q / np.linalg.norm( x - x_q, axis=1 ) )

def du_s_G(x,n,domain_index,result):
    '''
    Normal derivate for the singular component.
    '''    
    global q,x_q,ep_m,C
    result[:] = -C/(4.*np.pi*ep_m)  * np.sum( np.dot( x-
                            x_q , n)  * q / np.linalg.norm( x - x_q, axis=1 )**3 )
    
# EXPLICAR ESTAS FUNCIONES ----------------------------------
def q_times_G_L(x, n, domain_index, result):
    global q,x_q,ep_m,C , k
    result[:] = 1. / (4.*np.pi*ep_m)  * np.sum( q  / np.linalg.norm( x - x_q, axis=1 ) )
    
def Integral_superficie( A , B ):
    A = A.real.coefficients
    B = B.real.coefficients
    Area_F = open( os.path.join(path,'triangleAreas_'+str(mesh_density)+suffix+'.txt')).readlines()
    Areas  = np.zeros(len(Area_F))
    for i in range(len(Area_F)):
        Areas[i] = float(Area_F[i][:-1]) 
    return A*B*Areas
    
def u_h_Solver(dirichl_space, neumann_space, dual_to_dir_s):
    '''
    Solves for the harmonic component.
    grid    : Bempp atributte of grid
    space   : Piecewise or continuous polynomial space ('DP' or 'P')
    order   : Order of the space
    '''

    # Let's build u_s and du_s in the boundary
    u_s  = bempp.api.GridFunction(dirichl_space, fun=u_s_G)
    du_s = bempp.api.GridFunction(neumann_space, fun=du_s_G) 

    # Now lets create the asociated operators with the spaces defined above
    #
    # identity : I  : Identity matrix
    # dlp_in : K_in : Double-Layer operator for inner region
    # slp_in : V_in : Single-Layer operator for inner region
    # _out :          Same operators but for outer region, with kappa = 0.125
    identity = sparse.identity(     dirichl_space, dirichl_space, dual_to_dir_s)
    slp_in   = laplace.single_layer(neumann_space, dirichl_space, dual_to_dir_s)
    dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dual_to_dir_s)

    
    # The asociated equation is V_in du_s = (1/2+K_in)u_h = -(1/2+K_in)u_s (BC)
    sol, info,it_count = bempp.api.linalg.gmres( slp_in, -(dlp_in+0.5*identity)*u_s , return_iteration_count=True, tol=1e-4)
    print("The linear system for du_h was solved in {0} iterations".format(it_count))

    u_h = -u_s
    du_h = sol

    return u_h , du_h , it_count

def u_r_Solver(du_s , du_h , dirichl_space, neumann_space, dual_to_dir_s):
    '''
    Solves for the regular component.
    du_s  : Solution (Analytical) for the singular component
    du_h  : Solution for the harmonic component
    '''
    global k

    # Now lets create the asociated operators with the spaces defined above
    #
    # identity : I  : Identity matrix
    # dlp_in : K_in : Double-Layer operator for inner region
    # slp_in : V_in : Single-Layer operator for inner region
    # _out :          Same operators but for outer region, with kappa = 0.125
    
    identity = sparse.identity(     dirichl_space, dirichl_space, dual_to_dir_s)
    slp_in   = laplace.single_layer(neumann_space, dirichl_space, dual_to_dir_s)
    dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dual_to_dir_s)
    slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dual_to_dir_s, k)
    dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dual_to_dir_s, k)
    
    
    # The associated solution system is
    # | ( I/2 + K_L-in  )     (      -V_L-in     ) |  u_r  = 0
    # | ( I/2 - K_Y-out )     ( ep_m/ep_s V_Y-out) | du_r  = ep_m/ep_s V_Y-out*(du_s+du_h)  (BC)
    #
    # The left hand matrix is created
    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0, 0] = 0.5*identity + dlp_in
    blocked[0, 1] = -slp_in
    blocked[1, 0] = 0.5*identity - dlp_out
    blocked[1, 1] = (ep_m/ep_s)*slp_out

    # And the right hand matrix is built
    zero = bempp.api.GridFunction(dirichl_space, fun=zero_i)
    rhs = [ zero ,  -slp_out *(ep_m/ep_s)* (du_s+du_h)]

    # And the system is solved as
    sol, info,it_count = bempp.api.linalg.gmres( blocked , rhs, return_iteration_count=True, tol=1e-8)
    print("The linear system for u_r and du_r was solved in {0} iterations".format(it_count))
    u_r , du_r = sol
    
    return u_r, du_r, it_count

def adjoint_Solver( dirichl_space, neumann_space, dual_to_dir_s):
    '''
    Solves the adjoint equation
    '''
    global k    

    slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dual_to_dir_s, k)
    dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dual_to_dir_s, k)

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0, 0] = 0.5*identity + dlp_in
    blocked[0, 1] = -slp_in
    blocked[1, 0] = 0.5*identity - dlp_out
    blocked[1, 1] = (ep_m/ep_s)*slp_out

    zero = bempp.api.GridFunction(dirichl_space , fun=zero_i)
    P_GL = bempp.api.GridFunction(dirichl_space, fun=q_times_G_L)
    rs_r = [P_GL , zero]

    sol_r, info,it_count = bempp.api.linalg.gmres( blocked, rs_r , return_iteration_count=True, tol=1e-4)
    print("The linear system for phi was solved in {0} iterations".format(it_count))
    phi_r , dphi_r = sol_r
               
    return phi_r , dphi_r , it_count

def solvation_energy_trad( u_h , du_h , u_r , du_r , u_s , du_s , neumann_space , dirichl_space , x_q , q):
    '''
    Calculates the solvation energy in the traditional way using the harmonic,
    regular and singular solutions.
    '''

    # G_solv = Sum_i q_i *u_reac = Sum_i q_i * (u_h+u_r)  evaluated on each charge position.

    # The boundary potential operators are created
    slp_in_O = lp.single_layer(neumann_space, x_q.transpose()) 
    dlp_in_O = lp.double_layer(dirichl_space, x_q.transpose())

    # Now the potential is calculated at each point
    u_r_O = slp_in_O * du_r  -  dlp_in_O * u_r
    u_h_O = slp_in_O * du_h  +  dlp_in_O * u_s

    terms =  u_r_O + u_h_O

    # Adding constants we obtain the solvation energy
    K     = 0.5 * 4. * np.pi * 332.064 
    S     = K * np.sum(q * terms).real
    print("Three Term Splitting Solvation Energy : {:7.8f} [kCal/mol] ".format(S) )
    
    return S



